In [1]:
#Dependencies

from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt


In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")


In [6]:
# Reflect an existing database into a new model
#new_database = automap_base()

# Reflect the tables
#new_database.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
new_database.classes.keys()

[]

In [8]:
# Save references to each table
#station, date, prcp, tobs
measurement = new_database.classes.measurement
# station, name, latitude, longitude, elevation
weather_station = new_database.classes.station

AttributeError: measurement

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

last_date = session.query(measurement.date, measurement.prcp).order_by(measurement.date.desc()).first()[0]
entered_date = dt.datetime.strptime(last_date, '%Y-%m-%d')
entered_date = entered_date.date()

last_yr = entered_date - dt.timedelta(days=365)
last_yr

# Perform a query to retrieve the data and precipitation scores

one_year_prcp = session.query(measurement.date, measurement.prcp).filter(measurement.date >= last_yr).\
    filter(measurement.date <= last_date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column

df = pd.DataFrame(one_year_prcp, columns=['date', 'precipitation'])
df.set_index('date', inplace=True)

# Sort the dataframe by date

df = df.sort_values("date")
# Use Pandas Plotting with Matplotlib to plot the data

df.plot(rot=45)
plt.title("Date vs Precipitation")
plt.xlabel("Date")
plt.ylabel("Precipitation (inches)")

plt.savefig('Images/precipitation.png')
plt.show()

NameError: name 'measurement' is not defined

In [11]:
# Use Pandas to calcualte the summary statistics for the precipitation data

df.describe()

NameError: name 'df' is not defined

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_ct = session.query(measurement.station).distinct().count()
print("Station Count:", station_ct)

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?

active_stations = session.query(measurement.station, func.count(measurement.station)).\
    group_by(measurement.station).\
    order_by(func.count(measurement.station).desc()).all()

# List the stations and the counts in descending order.

active_stations_df = pd.DataFrame(active_stations, columns=['Station', 'Data Count'])
active_stations_df

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?

most_active_station = active_stations[0][0]

sel = [func.min(measurement.tobs), func.avg(measurement.tobs), func.max(measurement.tobs)]

most_active_station_tobs = session.query(*sel).\
    filter(measurement.station==most_active_station).all()[0]

print(f"Temperature Minimum: {most_active_station_tobs[0]}")
print(f"Temperature Average: {most_active_station_tobs[1]}")
print(f"Temperature Maximum: {most_active_station_tobs[2]}")

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
import datetime as dt
from pandas.plotting import table

results = session.query(measurement.tobs).\
    filter(measurement.date >= last_yr).\
    filter(measurement.date <= last_date).\
    filter(measurement.station==most_active_station).all()

df = pd.DataFrame(results, columns=['tobs'])
tobs_series.plot.hist(bins=12)
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.savefig('Images/station-histogram.png')
plt.tight_layout()
plt.show()
